# Imports

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
import math
import logging
import pandas as pd

from sentence_transformers import LoggingHandler, util
from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers.cross_encoder.evaluation import CEBinaryClassificationEvaluator
from sentence_transformers.readers import InputExample
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

/Users/lea.naccache/Desktop/FORMATION/maker-nlp/maker_nlp_env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Data loading

In [4]:
df = pd.read_csv("../data/train.csv").fillna("")
train_df, val_df = train_test_split(df, test_size=0.25, random_state=42)

# Training configuration

In [5]:
BATCH_SIZE = 16
EPOCHS = 4
MODEL_SAVE_PATH = "./saved_model"

# Data preparation

In [6]:
train_samples = []

for row in train_df.iterrows():
    train_samples.append(InputExample(texts=[row[1]["q1_text"], row[1]["q2_text"]], label=row[1]["is_duplicate"]))
    train_samples.append(InputExample(texts=[row[1]["q2_text"], row[1]["q1_text"]], label=row[1]["is_duplicate"]))

train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=BATCH_SIZE)  

In [7]:
val_samples = []
for row in val_df.iterrows():
    val_samples.append(InputExample(texts=[row[1]["q1_text"], row[1]["q2_text"]], label=row[1]["is_duplicate"]))

# Model

In [8]:
model = CrossEncoder('cross-encoder/ms-marco-TinyBERT-L-2', num_labels=1)

Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 612/612 [00:00<00:00, 69.7kB/s]
Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17.6M/17.6M [00:00<00:00, 17.7MB/s]
Downloading: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 543/543 [00:00<00:00, 484kB/s]
Downloading: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 232k/232k [00:00<00:00, 418kB/s]
Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 112/112 [00:00<00:00, 47.5kB/s]


In [9]:
evaluator = CEBinaryClassificationEvaluator.from_input_examples(val_samples, name='quora-val')

In [ ]:
warmup_steps = math.ceil(len(train_dataloader) * EPOCHS * 0.1)

model.fit(train_dataloader=train_dataloader,
          evaluator=evaluator,
          epochs=EPOCHS,
          evaluation_steps=5000,
          warmup_steps=warmup_steps,
          output_path=MODEL_SAVE_PATH)

Iteration:   0%|▍                                                                                                                        | 64/18951 [00:06<31:31,  9.99it/s]


Iteration:   1%|▊                                                                                                                       | 125/18951 [00:14<38:09,  8.22it/s]


Iteration:   1%|█▏                                                                                                                      | 188/18951 [00:21<32:36,  9.59it/s]


Iteration:   1%|█▋                                                                                                                      | 258/18951 [00:28<31:35,  9.86it/s]


Iteration:   2%|██▏                                                                                                                     | 349/18951 [00:37<31:24,  9.87it/s]


...

Iteration:  78%|████████████████████████████████████████████████████████████████████████████████████████████▎                         | 14820/18951 [29:39<07:04,  9.73it/s]


Iteration:  79%|████████████████████████████████████████████████████████████████████████████████████████████▋                         | 14883/18951 [29:45<07:13,  9.38it/s]


Iteration:  79%|█████████████████████████████████████████████████████████████████████████████████████████████                         | 14940/18951 [29:52<07:57,  8.39it/s]


Iteration:  79%|█████████████████████████████████████████████████████████████████████████████████████████████▎                        | 14993/18951 [29:58<07:39,  8.61it/s]


Iteration:  79%|█████████████████████████████████████████████████████████████████████████████████████████████▍                        | 14999/18951 [29:59<07:22,  8.92it/s]